In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [38]:
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
Y_train = pd.read_csv("y_train.csv")
Y_test = pd.read_csv("y_test.csv")
Y_train = pd.factorize(Y_train["Rating as Factor"])[0]
Y_test = pd.factorize(Y_test["Rating as Factor"])[0]

[0 0 0 ... 1 9 4]


In [33]:
pipe = Pipeline[('scaler', StandardScaler()),
               ('log_reg', sm.Logit(endog=Y_train, exog=sm.add_constant(X_train)).fit())]

NameError: name 'StandardScaler' is not defined

In [34]:
sm.Logit(endog=Y_train, exog=sm.add_constant(X_train)).fit()

ValueError: endog and exog matrices are different sizes